In [3]:
# AWDB loader

from datetime import datetime

import requests
import pandas as pd

API_DOMAIN = "https://api.snowdata.info/"

def get_awdb_data(site_ids, element="WTEQ", sdate=datetime(1899,10,1), edate=datetime.now(), orient="records", server=API_DOMAIN, sesh=None):
    """
    Takes a list of site ids or a single site id and by default returns SWE period of record data as a single or list of dataframes,
    but user can pass args to modify data returned.
    
    Valid elements include WTEQ, SNWD, PREC, SMS, STO, TAVG
    site_id takes the form of a triplet made from <network_site_id>:<state_abbrv>:<network> where network is either SNTL or MNST
    """
    dfs = []
    return_single = False
    if not isinstance(site_ids, list):
        site_ids = [site_ids]
        return_single = True
    for site_id in site_ids:
        endpoint = "data/getDaily"
        date_args = f"sDate={sdate:%Y-%m-%d}&eDate={edate:%Y-%m-%d}"
        frmt_args = f"format=json&orient={orient}"
        all_args = f"?triplet={site_id}&{date_args}&element={element}&{frmt_args}"
        url = f"{server}{endpoint}{all_args}"
        print(f"getting data for {site_id} {element} starting {sdate:%Y-%m-%d} and ending {edate:%Y-%m-%d}")
        data_col_lbl = f"{site_id}" + ":" + f"{element}"
        if sesh:
            req = sesh.get(url)
        else:
            req = requests.get(url)
        if req.ok:
            df = pd.DataFrame.from_dict(req.json())
            df.columns = ["Date", data_col_lbl]
            df.set_index("Date", inplace=True)
        else:
            print("  No data returned!")
            df = pd.DataFrame(
                data=[
                    {"Date": pd.NaT, data_col_lbl: None}
                ],
            ).set_index("Date").dropna()
        dfs.append(df)
    if return_single:
        return dfs[0]
    return dfs
    

df = get_awdb_data(site_ids="1099:UT:SNTL", sdate=datetime(2021, 10, 1))
df

getting data for 1099:UT:SNTL WTEQ starting 2021-10-01 and ending 2022-07-13


,1099:UT:SNTL:WTEQ
Date,
2021-10-01,0.0
2021-10-02,0.0
2021-10-03,0.0
2021-10-04,0.0
2021-10-05,0.0
...,...
2022-07-09,0.0
2022-07-10,0.0
2022-07-11,0.0


In [2]:
with requests.Session() as sesh:
    dfs = get_awdb_data(site_ids=["1099:UT:SNTL", "766:UT:SNTL"], sdate=datetime(2021, 10, 1), sesh=sesh)
pd.concat(dfs)

getting data for 1099:UT:SNTL WTEQ starting 2021-10-01 and ending 2022-07-13
getting data for 766:UT:SNTL WTEQ starting 2021-10-01 and ending 2022-07-13


,1099:UT:SNTL:WTEQ,766:UT:SNTL:WTEQ
Date,,
2021-10-01,0.0,NaN
2021-10-02,0.0,NaN
2021-10-03,0.0,NaN
2021-10-04,0.0,NaN
2021-10-05,0.0,NaN
...,...,...
2022-07-09,NaN,0.0
2022-07-10,NaN,0.0
2022-07-11,NaN,0.0
